In [1]:
print('hello')

hello


In [2]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

images_path = os.getcwd()

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    try:
        images = convert_from_path(pdf)
        for i in range(len(images)):
            images[i].save(images_path+'\\images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')
    except:
        pass

pdf_path = 'C:\\pocs\\riya_travel_invoices\\pdfs'

# store all the file names in this list
filelist = []

for root, dirs, files in os.walk(pdf_path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)
    except:
        pass

In [3]:
#################### parse output of original images ######################

import glob

for image in glob.glob('C:\\pocs\\riya_travel_invoices\\images\\*'):
    filename = os.path.split(image)[-1].split('.jpg')[0]
    psm_values = [4]
    for psm in psm_values:
        parsed_output = 'C:\\pocs\\riya_travel_invoices\\output\\'+filename+'_'+str(psm)
        command = 'tesseract {} {} -l eng --psm {}'.format(image, parsed_output, psm)
        os.system(command)

In [4]:
def get_traveler_name(text):
    pass

In [5]:
for file in glob.glob('C:\pocs\\riya_travel_invoices\output\*.txt'):

    with open(file) as fp:

        filename = os.path.split(file)[-1]
        words = filename.split('_')
        for i in range(len(words)):
            if 'page' in words[i]:
                ind = i
                break
        filename = '_'.join(words[:ind])
        # print(filename)

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            dl_number = get_traveler_name(text)


In [ ]:
import pdfplumber

file = 'C:\pocs\\riya_travel_invoices\pdfs\Bulk_CreditMemo_Air - XEBIA IT ARCHITECTS INDIA  GGN.pdf'

pdf = pdfplumber.open(file)
#specify pages
page = pdf.pages[0]
text = page.extract_text()

print(text, end='\n')

In [58]:
import tabula
import pandas as pd
import camelot

file = 'C:\pocs\\riya_travel_invoices\pdfs\Bulk_Sales_Invoice_Air - XEBIA IT ARCHITECTS INDIA  GGN.pdf'

t = tabula.read_pdf(file, pages='all', multiple_tables=True)
d = camelot.read_pdf(file, pages='all')


In [51]:
for i in range(len(t)):
    if 'Passenger' in t[i].columns:
        print(t[i])
        print()

In [63]:
for i in range(len(t)):

    if 'Sector' in t[i].columns:
        print(list(t[i]['Sector']))

[nan, 'DEL-AMS', 'AMS-DEL']
[nan, 'DEL-MAA', nan]
[nan, 'MAA-DEL', nan]
[nan, 'DEL-BLR', nan]
[nan, 'BLR-DEL', nan]
[nan, 'DEL-CDG', 'CDG-DEL']
[nan, 'DEL-AUH']
[nan, 'AUH-DEL']
[nan, 'DEL-UDR']
[nan, 'UDR-DEL']
[nan, 'BLR-MAA', nan]
[nan, 'MAA-BLR', nan]
[nan, 'HYD-BLR', nan, nan]
[nan, 'BLR-HYD', nan, nan]
[nan, 'BLR-MAA', nan]
[nan, 'MAA-BLR', nan]


In [128]:
def get_traveler_name(t):

    all_travelers = []
    for i in range(len(t)):
        l = []
        traveler_name = ''
        flag = 0
        if 'Sector' in t[i].columns:
            for col in t[i].columns:
                if 'Passenger' in col:
                    l = list(t[i][col])
                    # print('list---> ', l)

                    for k in range(len(l)):
                        if 'name' not in str(l[k]).lower() and 'ticket' not in str(l[k]).lower():
                            if l[k].strip() == 'MR':
                              traveler_name += ' '+str(l[k])  
                            else:
                                traveler_name += str(l[k])
                        if 'ticket' in str(l[k]).lower() or k==len(l)-1:
                            # traveler_name = traveler_name.replace('', '')
                            # print('name---> ', traveler_name)
                            all_travelers.append(traveler_name)
                            flag = 1
                            break
                        
                if flag:
                    break
    return all_travelers

def get_sectors(t):

    source_cities = []
    dest_cities = []

    for i in range(len(t)):
        if 'Sector' in t[i].columns:
            l = list(t[i]['Sector'])
            for n in l:
                if '-' in str(n):
                    source_cities.append((n.split('-')[0]).strip())
                    dest_cities.append((n.split('-')[-1]).strip())
                    # break

    return source_cities, dest_cities

def get_cities(city_codes):
    
    cities = []
    countries = []

    data = pd.read_excel('World_DB.xlsx')
    
    try:
        for city_code in city_codes:
            
                # print(code)
                value = data[data['iata'] == city_code.strip().upper()].iloc[0]
                # print('value---> ', value)
                cities.append(value['city'])
                countries.append(value['country'])
    except Exception as e:
        cities.append('None') 
        countries.append('None')

    return cities, countries

def get_date(date):
    formats = ['%d-%b-%Y']
    
    for date_format in formats:
        try:
            date = datetime.strptime(str(date), date_format)
            return date
        except ValueError:
            continue
    return False

def get_travel_date(t):
    
    date_col = []
    dates = []
    inner = []
    for i in range(len(t)):
        # print(t[i])
        
        if 'Travel Date' in t[i].columns:
            date_col = list(t[i]['Travel Date'])

            # print(date_col)
            for d in date_col:
                if '-' in str(d):
                    inner.append(d)
            dates.append(inner)
        inner = []
    return dates
        
    
        # for d in date_col:
        #     date = get_date(d)
        #     print(date, type(date))
        #     # if date:
        #     #     print(date)
        #     #     break 

def get_total_amount(text):
    
    for i in range(len(text)):
        # print(text[i])
        if 'total amount' in text[i].lower():
            # print(text[i])
            return float(text[i].split()[-1])

    # for i in range(len(t)):
        # print(t[i])

In [131]:
import glob
import os
from datetime import datetime
import tabula
import pandas as pd

traveler_names_sheet = []
travel_dates_sheet = []
source_cities_sheet = []
dest_cities_sheet = []
source_countries_sheet = []
dest_countries_sheet = []

nested_travel_dates = []

for file in glob.glob('C:\pocs\\riya_travel_invoices\pdfs\*'):

    t = tabula.read_pdf(file, pages='all', multiple_tables=True)

    with open(file) as fp:

        # filename = os.path.split(file)[-1]
        # words = filename.split('_')
        # for i in range(len(words)):
        #     if 'page' in words[i]:
        #         ind = i
        #         break
        # filename = '_'.join(words[:ind])
        # # print(filename)

        # text = fp.read()
        # text = text.splitlines()
        # text = [line for line in text if not line.isspace() and len(line) > 0]
        filename = os.path.split(file)[-1].split('.pdf')[0]
        # print(filename)
        
        traveler_name = get_traveler_name(t)
        source_sectors, dest_sectors = get_sectors(t)
        source_cities, source_countries = get_cities(source_sectors)
        dest_cities, dest_countries = get_cities(dest_sectors)
        travel_dates = get_travel_date(t)
        nested_travel_dates.extend(travel_dates)
        

        ## multiply traveler names for records that have multiple travel dates
        new_traveler_names = []
        for l in range(len(travel_dates)):
                new_traveler_names.extend([traveler_name[l]]*len(travel_dates[l]))
        # print(travel_dates)
        new_travel_dates = [item for subarray in travel_dates for item in subarray]


        # print(new_traveler_names)
        # print(new_travel_dates)
        # print(source_cities)
        # print(dest_cities)

        traveler_names_sheet.extend(new_traveler_names)
        travel_dates_sheet.extend(new_travel_dates)
        source_cities_sheet.extend(source_cities)
        source_countries_sheet.extend(source_countries)
        dest_cities_sheet.extend(dest_cities)
        dest_countries_sheet.extend(dest_countries)

total_amount_sheet = []
all_amounts = {}
for file in glob.glob('C:/pocs/riya_travel_invoices/output/*'):
    
    filename = os.path.split(file)[-1]
    words = filename.split('_')
    for i in range(len(words)):
        if 'page' in words[i]:
            ind = i
            break
    filename = '_'.join(words[:ind])
    # print(filename)

    with open(file) as f:

        text = f.read()
        text = text.splitlines()
        text = [line for line in text if not line.isspace() and len(line) > 0]
        # print(' '.join(text))

        if 'insurance' not in ' '.join(text).lower():
            total_amount = get_total_amount(text)
            # print(total_amount)

            # if total_amount:
            all_amounts[filename] = total_amount

# print(nested_travel_dates)
print(all_amounts)
                
# new_amounts = []
# for l in range(len(nested_travel_dates)):
#     if all_amounts[l]:
#         new_amounts.extend([all_amounts[l]/len(nested_travel_dates[l])]*len(nested_travel_dates[l]))
#     else:
#         new_amounts.append(None)
#     # print(new_amounts)

# print(len(new_amounts))
# print(new_amounts)

{'Bulk_CreditMemo_Air_-_XEBIA_IT_ARCHITECTS_INDIA__GGN': None, 'Bulk_Sales_Invoice_Air_-_XEBIA_IT_ARCHITECTS_INDIA__GGN': 4876.6}


In [118]:
print(nested_travel_dates)
print(len(nested_travel_dates[-1]))

[['24-Jun-2024'], ['19-Jun-2024', '26-Jun-2024'], ['09-Jun-2024'], ['11-Jun-2024'], ['09-Jun-2024'], ['16-Jun-2024'], ['11-Jun-2024', '15-Jun-2024'], ['09-Jun-2024'], ['29-Jun-2024'], ['12-Jun-2024'], ['12-Jun-2024'], ['13-Jun-2024'], ['17-Jun-2024'], ['11-Jun-2024'], ['12-Jun-2024'], ['13-Jun-2024'], ['17-Jun-2024']]
1


In [109]:
print(len(all_amounts))

17


In [107]:
print(source_cities_sheet)

['Mumbai', 'New Delhi', 'Amsterdam', 'New Delhi', 'Chennai', 'New Delhi', 'Bangalore', 'New Delhi', 'Paris', 'New Delhi', 'Abu Dhabi', 'New Delhi', 'Udaipur', 'Bangalore', 'Chennai', 'Hyderabad', 'Bangalore', 'Bangalore', 'Chennai']


In [130]:
df = pd.DataFrame({
    'Traveler Name': traveler_names_sheet,
    'Travel Date': travel_dates_sheet,
    'Source City': source_cities_sheet,
    'Source Country': source_countries_sheet,
    'Destination City': dest_cities_sheet,
    'Destination Country': dest_countries_sheet,
    'Total Amount': new_amounts
}
)

df

,Traveler Name,Travel Date,Source City,Source Country,Destination City,Destination Country,Total Amount
0,PRAKASH/VARUN MR,24-Jun-2024,Mumbai,IN,New Delhi,IN,NaN
1,DASTIDAR/SHANKAR MR,19-Jun-2024,New Delhi,IN,Amsterdam,NL,44274.5
2,DASTIDAR/SHANKAR MR,26-Jun-2024,Amsterdam,NL,New Delhi,IN,44274.5
3,KUMAR/AKHILESH MR,09-Jun-2024,New Delhi,IN,Chennai,IN,6484.6
4,KUMAR/AKHILESH MR,11-Jun-2024,Chennai,IN,New Delhi,IN,3277.6
5,RANJAN/ASHISH MR,09-Jun-2024,New Delhi,IN,Bangalore,IN,4197.6
6,ASHISHRANJAN,16-Jun-2024,Bangalore,IN,New Delhi,IN,7519.6
7,GAMBHIR/SACHIN MR,11-Jun-2024,New Delhi,IN,Paris,FR,200.0
8,GAMBHIR/SACHIN MR,15-Jun-2024,Paris,FR,New Delhi,IN,200.0
9,ROHIT LNU,09-Jun-2024,New Delhi,IN,Abu Dhabi,AE,400.0
